**Check if NVIDIA GPU is enabled**

In [1]:
!nvidia-smi

Sun Mar 27 11:30:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**Connect google drive (personal and robomasters, sets up aliases for both)**

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/Shareddrives/TAMU\ Robomasters/RM\ -\ 2020-2021/Software/Computer\ Vision/Modeling\ / /RMdrive
# !ls /RMdrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
# !ls /mydrive

Mounted at /content/gdrive


In [3]:
! ls /RMdrive


 5_23_best.weights	      no_dji_best.weights
 5_23.weights		     'output(5).avi'
 Datasets		      result.nosync.mp4
'Modeling '		      v3t.names
'Modeling Meeting Log.gdoc'   yolov4_training-3.cfg
 new_yolov4.cfg		      yolov4_training_last.weights
 new_yolov4.weights	      yolov5


**Install Requirements**

In [4]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 12148, done.
remote: Total 12148 (delta 0), reused 0 (delta 0), pack-reused 12148
Receiving objects: 100% (12148/12148), 11.47 MiB | 25.32 MiB/s, done.
Resolving deltas: 100% (8439/8439), done.
/content/yolov5
     |████████████████████████████████| 596 kB 5.4 MB/s 
     |████████████████████████████████| 178 kB 7.6 MB/s 
     |████████████████████████████████| 1.1 MB 36.6 MB/s 
     |████████████████████████████████| 138 kB 50.0 MB/s 
     |████████████████████████████████| 67 kB 5.5 MB/s 
     |████████████████████████████████| 145 kB 27.7 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 

In [5]:
 # Create folder on google drive so that we can save there the weights
!mkdir -p "/mydrive/yolov5"

In [6]:
# Extra images
!apt-get install p7zip-full
# Downloading DJI dataset
!7za e /RMdrive/Datasets/yolo_no_disabled_no_base.zip -odata/obj

# Use DJI dataset
import glob
images_list = glob.glob("data/obj/*.jpg")
print(len(images_list))

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-6).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.

7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /RMdrive/Datasets/                            1 file, 10412480841 bytes (9931 MiB)

Extracting archive: /RMdrive/Datasets/yolo_no_disabled_no_base.zip
  1% 256 Open               2% 512 Open               3% 768 Open               7% 1536 Open                9% 2048 Open               11% 2304 Open               12% 25

In [ ]:
# For DJI dataset ONLY
import os, random

os.mkdir('test')
os.mkdir('test/images')
os.mkdir('test/labels')
os.mkdir('train')
os.mkdir('train/images')
os.mkdir('train/labels')
os.mkdir('valid')
os.mkdir('valid/images')
os.mkdir('valid/labels')

# Assign images to train/test/valid
for fileName in images_list:
    fileName = fileName.rsplit('/', 1)[-1]
    fileName = fileName.rsplit('.', 1)[0]
    rand = random.uniform(0, 1)

    if rand <= 0.15:
        os.rename("data/obj/" + fileName + ".jpg", "valid/images/" + fileName + ".jpg")
        os.rename("data/obj/" + fileName + ".txt", "valid/labels/" + fileName + ".txt")
    elif rand <= 0.3:
        os.rename("data/obj/" + fileName + ".jpg", "test/images/" + fileName + ".jpg")
        os.rename("data/obj/" + fileName + ".txt", "test/labels/" + fileName + ".txt")
    else:
        os.rename("data/obj/" + fileName + ".jpg", "train/images/" + fileName + ".jpg")
        os.rename("data/obj/" + fileName + ".txt", "train/labels/" + fileName + ".txt")

print(images_list)

['data/obj/AllianceVsArtisans_BO2_2_102.jpg', 'data/obj/TDTVsI Hiter_BO5_4_279.jpg', 'data/obj/yxsVsXRC_HG_BO3_1_219.jpg', 'data/obj/EvolutionVsHuanan_tiger_BO3_2_78.jpg', 'data/obj/VW_CH3RoboGrinderVsJiaoDrangon_BO2_2_101.jpg', 'data/obj/WMJVsWolfTooth_BO2_2_140.jpg', 'data/obj/WMJVsWolfTooth_BO2_2_325.jpg', 'data/obj/EvolutionVsHuanan_tiger_BO3_2_187.jpg', 'data/obj/SJWDXVsHLL_BO2_2_317.jpg', 'data/obj/LingzhiTechAresVsThunder_BO2_2_134.jpg', 'data/obj/TDTVsI Hiter_BO5_3_338.jpg', 'data/obj/zhongweiVsRobotPilots_BO3_1_43.jpg', 'data/obj/VW_CH3RoboGrinderVsJiaoDrangon_BO2_1_30.jpg', 'data/obj/VW_CH3ZwdlVsAlliance_BO2_2_180.jpg', 'data/obj/VW_CH3TDTVsHERO_BO2_2_29.jpg', 'data/obj/SPRVsTOE_BO5_3_109.jpg', 'data/obj/EvolutionVsHuanan_tiger_BO3_2_396.jpg', 'data/obj/WMJVsWolfTooth_BO2_2_52.jpg', 'data/obj/SJWDXVsHLL_BO2_2_359.jpg', 'data/obj/GjhyVsHuanan_tiger_BO5_2_45.jpg', 'data/obj/FireWireVsBorn of Fire_BO2_1_331.jpg', 'data/obj/EvolutionVsHuanan_tiger_BO3_2_395.jpg', 'data/obj/TDTVsI

In [ ]:
!ls /RMdrive/Datasets/

'DJI Dataset(Chinese)'	     image
'DJI dataset(ENGLISH-SSD)'  'Test videos'
'DJI ROCO.zip'		     yolo_eng_all.zip
 eng_all_separated.zip	     yolo_no_disabled_no_base.zip


In [ ]:
!pwd

/content/yolov5


In [ ]:
!ls

CONTRIBUTING.md  Dockerfile  LICENSE	requirements.txt  tutorial.ipynb
data		 export.py   models	setup.cfg	  utils
detect.py	 hubconf.py  README.md	train.py	  val.py


In [ ]:
# Downloading Twitch dataset
!curl -L "https://app.roboflow.com/ds/Q0UPOHMq2X?key=YMyflpkxjc" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   888  100   888    0     0   1274      0 --:--:-- --:--:-- --:--:--  1272
100 36.2M  100 36.2M    0     0  27.2M      0  0:00:01  0:00:01 --:--:--  263M
Archive:  roboflow.zip
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
 extracting: data.yaml               
   creating: test/
   creating: test/images/
 extracting: test/images/GjhyVsHuanan_tiger_BO3_2_292_jpg.rf.cfe9c6f5cb36124a38b508a133e03d41.jpg  
 extracting: test/images/GjhyVsHuanan_tiger_BO5_2_53_jpg.rf.4fd73a58f11eb1310acd9c3b5fffcda6.jpg  
 extracting: test/images/IMG_RED_1_0000_jpg.rf.26063158fd91d7713baff3da727cb252.jpg  
 extracting: test/images/IMG_RED_1_0001_jpg.rf.efb91a51fd03afd1cd6efc7dec6036b4.jpg  
 extracting: test/images/IMG_RED_1_0002_jpg.rf.e5960ce17173d9891541a7b89dfb364c.jpg  
 extracting: test/images/IMG_RED_1_0003_jpg.r

In [ ]:
!ls

CONTRIBUTING.md  hubconf.py	      requirements.txt	utils
data		 LICENSE	      setup.cfg		valid
data.yaml	 models		      test		val.py
detect.py	 README.dataset.txt   train
Dockerfile	 README.md	      train.py
export.py	 README.roboflow.txt  tutorial.ipynb


In [ ]:
yaml_data = {}
import yaml
with open('data.yaml', 'r') as file:
	data = yaml.safe_load(file)
	data['train'] = './train/images'
	data['val'] = './valid/images'
	yaml_data = data

with open('data.yaml','w') as file:
	yaml.dump(yaml_data,file)
 
 
	


In [ ]:
!cat data.yaml

names:
- '0'
- '1'
nc: 2
train: ./train/images
val: ./valid/images


In [ ]:
!pip install pyngrok
from pyngrok import ngrok# Open a HTTP tunnel on port 8090
public_url = ngrok.connect(port = '8090')

public_url

     |████████████████████████████████| 745 kB 4.0 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19006 sha256=035854794a033cea3b9b19983e4a1e66e1003c3b61dc9c6579882bbdec09a1f7
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


<NgrokTunnel: "http://c130-104-155-209-193.ngrok.io" -> "http://localhost:80">

In [ ]:
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="ultralytics")

upload and label your dataset, and get an API KEY here: https://app.roboflow.com/?model=yolov5&ref=ultralytics


In [ ]:
# set up environment
os.environ["DATASET_DIRECTORY"] = "data/obj"

In [ ]:
#after following the link above, recieve python code with these fields filled in
#from roboflow import Roboflow
#rf = Roboflow(api_key="YOUR API KEY HERE")
#project = rf.workspace().project("YOUR PROJECT")
#dataset = project.version("YOUR VERSION").download("yolov5")

In [ ]:
!ls data/obj

ls: cannot access 'data/obj': No such file or directory


# Step 3: Train Our Custom YOLOv5 model

Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** Our dataset locaiton is saved in the `dataset.location`
- **weights:** specify a path to weights to start transfer learning from. Here we choose the generic COCO pretrained checkpoint.
- **cache:** cache images for faster training

need to convert YOLO Darknet text to YOLOV5 pytorch text

In [ ]:
!cat data/train.txt

cat: data/train.txt: No such file or directory


In [ ]:
!pwd

/content/yolov5


In [ ]:
!ls /RMdrive

ls: cannot access '/RMdrive': No such file or directory


In [ ]:
!python train.py --resume /mydrive/yolov5/exp37/weights/last.pt

train: weights=yolov5s.pt, cfg=, data=data/coco128.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=300, batch_size=16, imgsz=640, rect=False, resume=/mydrive/yolov5/exp37/weights/last.pt, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
Resuming training from /mydrive/yolov5/exp37/weights/last.pt
YOLOv5 🚀 v6.0-159-gdb6ec66 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anch

In [ ]:
!python train.py --img 416 --batch 16 --epochs 3000 --data data.yaml --weights yolov5s.pt --cache --project /mydrive/yolov5 --name exp

train: weights=yolov5s.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=3000, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=/mydrive/yolov5, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-159-gdb6ec66 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0.5

# Evaluate Custom YOLOv5 Detector Performance
Training losses and performance metrics are saved to Tensorboard and also to a logfile.

If you are new to these metrics, the one you want to focus on is `mAP_0.5` - learn more about mean average precision [here](https://blog.roboflow.com/mean-average-precision/).

In [ ]:
!python detect.py --weights /mydrive/yolov5/exp37/weights/best.pt --img 416 --conf 0.5 --source vids --project /mydrive/yolov5 --name exp2

Streaming output truncated to the last 5000 lines.
video 1/2 (3122/6963) /content/yolov5/vids/comp_2021__group_vs_ucsd__soldier__round_1.avi: 256x416 Done. (0.117s)
video 1/2 (3123/6963) /content/yolov5/vids/comp_2021__group_vs_ucsd__soldier__round_1.avi: 256x416 Done. (0.118s)
video 1/2 (3124/6963) /content/yolov5/vids/comp_2021__group_vs_ucsd__soldier__round_1.avi: 256x416 Done. (0.115s)
video 1/2 (3125/6963) /content/yolov5/vids/comp_2021__group_vs_ucsd__soldier__round_1.avi: 256x416 Done. (0.113s)
video 1/2 (3126/6963) /content/yolov5/vids/comp_2021__group_vs_ucsd__soldier__round_1.avi: 256x416 Done. (0.123s)
video 1/2 (3127/6963) /content/yolov5/vids/comp_2021__group_vs_ucsd__soldier__round_1.avi: 256x416 Done. (0.115s)
video 1/2 (3128/6963) /content/yolov5/vids/comp_2021__group_vs_ucsd__soldier__round_1.avi: 256x416 Done. (0.121s)
video 1/2 (3129/6963) /content/yolov5/vids/comp_2021__group_vs_ucsd__soldier__round_1.avi: 256x416 Done. (0.126s)
video 1/2 (3130/6963) /content/yolov5

**Get mAP**

In [ ]:
!python val.py --task test --weights /mydrive/yolov5/exp37/weights/best.pt --img 416 --data data.yaml --project /mydrive/yolov5 --name map

val: data=data.yaml, weights=['/mydrive/yolov5/exp37/weights/best.pt'], batch_size=32, imgsz=416, conf_thres=0.001, iou_thres=0.6, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.0-159-gdb6ec66 torch 1.10.0+cu111 CPU

Fusing layers... 
Model Summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning 'test/labels' images and labels...1562 found, 0 missing, 14 empty, 0 corrupted: 100% 1562/1562 [00:05<00:00, 260.66it/s]
test: New cache created: test/labels.cache
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 49/49 [04:00<00:00,  4.92s/it]
                 all       1562      17733      0.919      0.882      0.927      0.516
                   0       1562       8667      0.918      0.886      0.927      0.515
                   1       1562      

In [ ]:
!ls test/images

image808_png_jpg.rf.141b9cbf5a86f72a13374f8a0a20a880.jpg
image809_png_jpg.rf.48146a5b3d549acbb8873c7fa488cb59.jpg
image810_png_jpg.rf.64da6947d9ee55fa0a19ff43d99bccbe.jpg
image811_png_jpg.rf.fdb5102dbeabd36b903e8d9eb687c80a.jpg
image812_png_jpg.rf.1256614ad575622eb9aefc405b5a08e4.jpg
image813_png_jpg.rf.3e931b4b1d988bc30bc030a45ebe47a4.jpg
image814_png_jpg.rf.61137f9da76cb3ea93da571635b51813.jpg
image815_png_jpg.rf.839b109121c62cba74e469b8dbd6e682.jpg
image816_png_jpg.rf.c0bb9f5cea1d0167f1c1dbc677666035.jpg
image817_png_jpg.rf.4aa3dd47b5cfddf5c7c8e5bab2d571b8.jpg
image818_png_jpg.rf.90e81b8004da017274b326373740c50a.jpg
image819_png_jpg.rf.f4d250cd52b31dc80cbd162a994b4c97.jpg
image820_png_jpg.rf.501451f61873d878006b44eb47e21a1e.jpg
image821_png_jpg.rf.c7a36c2eaea5da4a89ca8b9cbd93c615.jpg
image822_png_jpg.rf.864ba1aabc8af22b477284b1235ede34.jpg
image823_png_jpg.rf.8de79550942e52f5a273f2f69046d161.jpg
image824_png_jpg.rf.36fa579e420e99dea43cdd3505f35bce.jpg
image825_png_jpg.rf.44a9ba994a9